In [253]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import ast

In [260]:
df = pd.read_csv('../rounds_data.csv')
df.head()

,player_1_team_name,player_1_name,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_inventory,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,...,player_10_team_num,player_10_kills_total,player_10_deaths_total,player_10_inventory,player_10_has_defuser,player_10_has_helmet,player_10_armor_value,player_10_current_equip_value,player_10_balance,round_winner
0,G2 Esports,NiKo,2.0,0,0,"['knife_m9_bayonet', 'Glock-18', 'Smoke Grenade']",False,False,100.0,1150.0,...,3.0,0.0,0.0,"['knife_flip', 'Dual Berettas']",False,False,100.0,950.0,150.0,NaN
1,G2 Esports,NiKo,2.0,1,1,"['knife_m9_bayonet', 'Desert Eagle']",False,False,0.0,700.0,...,3.0,0.0,1.0,"['knife_flip', 'USP-S', 'Smoke Grenade', 'Ince...",False,True,100.0,3550.0,50.0,CT
2,G2 Esports,NiKo,2.0,1,2,"['knife_m9_bayonet', 'Glock-18', 'Galil AR', '...",False,True,100.0,4100.0,...,3.0,0.0,2.0,"['knife_flip', 'USP-S', 'Flashbang', 'MP9', 'S...",False,True,100.0,3850.0,1100.0,CT
3,G2 Esports,NiKo,2.0,1,3,"['knife_m9_bayonet', 'Glock-18', 'AK-47', 'Smo...",False,True,100.0,5100.0,...,3.0,1.0,3.0,"['knife_flip', 'USP-S', 'M4A1-S', 'Smoke Grena...",False,False,100.0,4950.0,0.0,T
4,G2 Esports,NiKo,2.0,3,4,"['knife_m9_bayonet', 'Molotov', 'Flashbang', '...",False,True,100.0,2800.0,...,3.0,2.0,4.0,"['knife_flip', 'USP-S', 'MP9', 'Smoke Grenade'...",False,True,100.0,3650.0,100.0,CT


In [261]:
df.columns

Index(['player_1_team_name', 'player_1_name', 'player_1_team_num',
       'player_1_kills_total', 'player_1_deaths_total', 'player_1_inventory',
       'player_1_has_defuser', 'player_1_has_helmet', 'player_1_armor_value',
       'player_1_current_equip_value',
       ...
       'player_10_team_num', 'player_10_kills_total', 'player_10_deaths_total',
       'player_10_inventory', 'player_10_has_defuser', 'player_10_has_helmet',
       'player_10_armor_value', 'player_10_current_equip_value',
       'player_10_balance', 'round_winner'],
      dtype='object', length=111)

In [262]:
df.dropna(subset=['round_winner'], inplace=True)

In [263]:
df['round_winner'].unique()

array(['CT', 'T', '3', '2', '1'], dtype=object)

In [264]:
round_winner_counts = df['round_winner'].value_counts()
print(round_winner_counts)

round_winner
CT    1797
T     1705
2      428
3      356
1        5
Name: count, dtype: int64


In [265]:
df = df[df['round_winner'] != "1"]

In [266]:
df['round_winner'].count()

4286

In [267]:
df['round_winner'] = df['round_winner'].replace({'CT': 1, '3': 1, 'T': 0, '2': 0}).astype(int)

C:\Users\Cosmin\AppData\Local\Temp\ipykernel_20288\1361703144.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['round_winner'] = df['round_winner'].replace({'CT': 1, '3': 1, 'T': 0, '2': 0}).astype(int)


In [268]:
df['round_winner'].unique()

array([1, 0])

In [269]:
round_winner_counts = df['round_winner'].value_counts()
print(round_winner_counts)

round_winner
1    2153
0    2133
Name: count, dtype: int64


In [270]:
for i in range(1,11):
    print(df[f'player_{i}_team_num'].unique())

[2. 3.]
[3. 2.]
[3. 2.]
[3. 2.]
[2. 3.]
[2. 3.]
[3. 2.]
[ 2.  3. nan]
[2. 3.]
[3. 2.]


In [271]:
team_num_columns = [f'player_{i}_team_num' for i in range(1, 11)]
df = df.dropna(subset=team_num_columns)

In [272]:
for i in range(1,11):
    print(df[f'player_{i}_team_num'].unique())

[2. 3.]
[3. 2.]
[3. 2.]
[3. 2.]
[2. 3.]
[2. 3.]
[3. 2.]
[2. 3.]
[2. 3.]
[3. 2.]


In [273]:
for i in range(1, 11):
    df[f'player_{i}_team_num'] = df[f'player_{i}_team_num'].replace({2.0: 0, 3.0: 1}).astype(int)

In [274]:
for i in range(1,11):
    print(df[f'player_{i}_team_num'].unique())

[0 1]
[1 0]
[1 0]
[1 0]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[1 0]


In [275]:
for i in range(1, 11):
    for suffix in ['has_defuser', 'has_helmet']:
        column_name = f'player_{i}_{suffix}'
        df = df[df[column_name].isin([True, False])]
boolean_columns = [f'player_{i}_{suffix}' for i in range(1, 11) for suffix in ['has_defuser', 'has_helmet']]
df[boolean_columns] = df[boolean_columns].astype(int)

In [276]:
primary_weapons = [
    'AK-47', 'M4A1-S', 'M4A4', 'AWP', 'AUG', 'SG 553', 'FAMAS', 'Galil AR',
    'Scar-20', 'G3SG1', 'MP5-SD', 'UMP-45', 'P90', 'MAC-10', 'MP7', 'MP9',
    'PP-Bizon', 'Negev', 'M249', 'Nova', 'XM1014', 'Sawed-Off', 'MAG-7'
]
secondary_weapons = [
    'P2000', 'USP-S', 'Glock-18', 'P250', 'Five-SeveN', 'Tec-9', 
    'CZ75-Auto', 'Desert Eagle', 'R8 Revolver', 'Dual Berettas'
]

grenades = ['Smoke Grenade', 'High Explosive Grenade', 'Flashbang', 'Incendiary Grenade', 'Molotov', 'Decoy Grenade']

In [277]:
new_columns = []
for i in range(1, 11):
    for weapon in primary_weapons + secondary_weapons:
        col_name = f'player_{i}_{weapon}'
        new_columns.append(pd.Series(0, index=df.index, name=col_name))
    for grenade in grenades:
        col_name = f'player_{i}_{grenade}'
        new_columns.append(pd.Series(0, index=df.index, name=col_name))
df = pd.concat([df] + new_columns, axis=1)

In [278]:
def convert_to_list(x):
    try:
        return ast.literal_eval(x)
    except ValueError:
        return []  
    except SyntaxError:
        return []

In [279]:
for i in range(1, 11):
    column_name = f'player_{i}_inventory'
    df[column_name] = df[column_name].apply(convert_to_list)

In [280]:
def one_hot_encode_inventory(row):
    for i in range(1, 11):
        inventory = row[f'player_{i}_inventory']
        if isinstance(inventory, list):
            for item in inventory:
                if item in primary_weapons or item in secondary_weapons:
                    row[f'player_{i}_{item}'] = 1
                elif item in grenades:
                    row[f'player_{i}_{item}'] = 1
    return row

In [281]:
df = df.apply(one_hot_encode_inventory, axis=1)

In [282]:
for i in range(1, 11):
    df.drop(columns=[f'player_{i}_inventory'], inplace=True)

In [283]:
df.head()

,player_1_team_name,player_1_name,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,player_1_balance,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
1,G2 Esports,NiKo,0,1,1,0,0,0.0,700.0,1650,...,0,0,0,0,1,0,1,1,0,0
2,G2 Esports,NiKo,0,1,2,0,1,100.0,4100.0,150,...,0,0,0,0,1,1,1,1,0,0
3,G2 Esports,NiKo,0,1,3,0,1,100.0,5100.0,1000,...,0,0,0,0,1,1,0,1,0,0
4,G2 Esports,NiKo,0,3,4,0,1,100.0,2800.0,400,...,0,0,0,0,1,1,0,1,0,0
5,G2 Esports,NiKo,0,4,5,0,0,0.0,200.0,3100,...,0,0,0,0,1,1,1,1,0,0


In [284]:
df.columns

Index(['player_1_team_name', 'player_1_name', 'player_1_team_num',
       'player_1_kills_total', 'player_1_deaths_total', 'player_1_has_defuser',
       'player_1_has_helmet', 'player_1_armor_value',
       'player_1_current_equip_value', 'player_1_balance',
       ...
       'player_10_CZ75-Auto', 'player_10_Desert Eagle',
       'player_10_R8 Revolver', 'player_10_Dual Berettas',
       'player_10_Smoke Grenade', 'player_10_High Explosive Grenade',
       'player_10_Flashbang', 'player_10_Incendiary Grenade',
       'player_10_Molotov', 'player_10_Decoy Grenade'],
      dtype='object', length=491)

In [285]:
df = df.drop(columns=[f'player_{i}_{suffix}' for i in range(1, 11) for suffix in ['team_name', 'name']])

In [286]:
df.head()

,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,player_1_balance,player_2_team_num,player_2_kills_total,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
1,0,1,1,0,0,0.0,700.0,1650,1,1.0,...,0,0,0,0,1,0,1,1,0,0
2,0,1,2,0,1,100.0,4100.0,150,1,4.0,...,0,0,0,0,1,1,1,1,0,0
3,0,1,3,0,1,100.0,5100.0,1000,1,5.0,...,0,0,0,0,1,1,0,1,0,0
4,0,3,4,0,1,100.0,2800.0,400,1,7.0,...,0,0,0,0,1,1,0,1,0,0
5,0,4,5,0,0,0.0,200.0,3100,1,8.0,...,0,0,0,0,1,1,1,1,0,0


In [287]:
X = df.drop(columns=['round_winner'])
y = df['round_winner']

In [288]:
X.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [289]:
X.head()

,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,player_1_balance,player_2_team_num,player_2_kills_total,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
1,0,1,1,0,0,0.0,700.0,1650,1,1.0,...,0,0,0,0,1,0,1,1,0,0
2,0,1,2,0,1,100.0,4100.0,150,1,4.0,...,0,0,0,0,1,1,1,1,0,0
3,0,1,3,0,1,100.0,5100.0,1000,1,5.0,...,0,0,0,0,1,1,0,1,0,0
4,0,3,4,0,1,100.0,2800.0,400,1,7.0,...,0,0,0,0,1,1,0,1,0,0
5,0,4,5,0,0,0.0,200.0,3100,1,8.0,...,0,0,0,0,1,1,1,1,0,0


In [290]:
df['round_winner'].count()

4251

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [292]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [293]:
model = LogisticRegression(max_iter=1000)

In [294]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [295]:
y_pred = model.predict(X_test)

In [296]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.67      0.70       856
           1       0.69      0.75      0.72       845

    accuracy                           0.71      1701
   macro avg       0.71      0.71      0.71      1701
weighted avg       0.71      0.71      0.71      1701

[[573 283]
 [215 630]]
